<a href="https://colab.research.google.com/github/Falenbayu/-Proyek-Klasifikasi-Gambar-Batu-Gunting-Kertas-Falen/blob/main/Proyek_Akhir_Klasifikasi_Gambar_Rockpaperscissors_Falen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Name        : FALEN BAYU ADHETYA
## Username    : falen_bayu
## Email       : falenbayu@gmail.com

In [ ]:
import pandas as pd
from pandas import DataFrame

# Library to extract files
import os
import zipfile

# Library to display images
%matplotlib inline
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Library to using Image Generator
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files

In [ ]:

# download dataset using wget command
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

In [ ]:
local_zip = '/content/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
paper_folder = os.path.join('/content/rockpaperscissors/paper')

# Show the total number of images in the paper folder
len_paper_folder = len(os.listdir(paper_folder))
print("Total paper pictures:", len_paper_folder)

# Show 5 data in the paper folder
paper = os.listdir(paper_folder)
print("5 data in the paper folder:")
i=0
paper_temp = []
for file_paper in paper[:5]:
  paper_temp.append(file_paper)
  i=i+1
data_paper = {'File Name': paper_temp}
df_paper = pd.DataFrame(data_paper,
                           index=[1,2,3,4,5])
df_paper.head()

In [ ]:

image_paper = image.load_img('/content/rockpaperscissors/paper/CHf3O6SDtuPbbD0s.png')
image_paper_plot = plt.imshow(image_paper)

In [ ]:
rock_folder = os.path.join('/content/rockpaperscissors/rock')

# Show the total number of images in the rock folder
len_rock_folder = len(os.listdir(rock_folder))
print("Total rock pictures:", len_rock_folder)

# Show 5 data in the rock folder
rock = os.listdir(rock_folder)
print("5 data in the rock folder:")
i=0
rock_temp = []
for file_rock in rock[:5]:
  rock_temp.append(file_rock)
  i=i+1
data_rock = {'File Name': rock_temp}
df_rock = pd.DataFrame(data_rock,
                           index=[1,2,3,4,5])
df_rock.head()

In [ ]:
image_rock = image.load_img('/content/rockpaperscissors/rock/yrzokMUYHlOLsRhQ.png')
image_plot = plt.imshow(image_rock)

In [ ]:
scissors_folder = os.path.join('/content/rockpaperscissors/scissors')

# Show the total number of images in the scissors folder
len_scissors_folder = len(os.listdir(scissors_folder))
print("Total scissors pictures:", len_scissors_folder)

# Show 5 data in the scissors folder
scissors = os.listdir(scissors_folder)
print("5 data in the scissors folder:")
i=0
scissors_temp = []
for file_scissors in scissors[:5]:
  scissors_temp.append(file_scissors)
  i=i+1
data_scissors = {'File Name': scissors_temp}
df_scissors = pd.DataFrame(data_scissors,
                           index=[1,2,3,4,5])
df_scissors.head()

In [ ]:
image_scissors = image.load_img('/content/rockpaperscissors/scissors/3kVoftgTNdDcvPPY.png')
image_plot = plt.imshow(image_scissors)

Menggunakan Image Generator

In [ ]:
train_dir = os.path.join("/content/rockpaperscissors/rps-cv-images/")

In [ ]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range=0.2,
                    zoom_range=0.2,
                    fill_mode = 'nearest',
                    # Data is divided into 40% data validation and 60% training data
                    validation_split=0.4)

In [ ]:

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=40,
        class_mode='categorical',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=40,
        class_mode='categorical',
        subset='validation')

Membuat Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
class stop(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.96):
      print("\nAccuracy has reached 0.96")
      self.model.stop_training=True
callbacks=stop()
# train models with model.fit
train_models = model.fit(
      train_generator,
      batch_size=15,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=5,
      callbacks=[callbacks])


In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)

  print(fn)
  if classes[0,0]!=0:
    print('This picture shows the shape of the paper')
  elif classes[0,1]!=0:
    print('This picture shows the shape of the rock')
  elif classes[0,2]!=0:
    print('This picture shows the shape of the scissors')
  else:
    print('Unknown image')